In [10]:
import os
import shutil

city = "manila"
extract = "https://s3.amazonaws.com/metro-extracts.mapzen.com/manila_philippines.osm.pbf"
output_data_dir = "/data/tiles/"


# need to use local file system to build tiles -- docker mounted volumes are too slow
tmp_build_dir = "/tmp/tiles/"

city_data_dir = tmp_build_dir + city + "/"

if not os.path.exists(output_data_dir):
    os.makedirs(output_data_dir)

output_tar = output_data_dir + city + ".tar"
    
if os.path.exists(output_tar):
    print "Output file " + output_tar + " already exists."
    break;


if os.path.exists(city_data_dir):
    shutil.rmtree(city_data_dir)
    
os.makedirs(city_data_dir)
os.chdir(city_data_dir)

# download some data and make tiles out of it
print "Downloading: " + extract
os.system("wget " + extract + " -O extract.pbf")

#get the config and setup for it
print "Building tile config..."
os.system("valhalla_build_config --mjolnir-tile-dir " + city_data_dir + "valhalla_tiles --mjolnir-tile-extract " + city_data_dir + "valhalla_tiles.tar --mjolnir-timezone " + city_data_dir + "valhalla_tiles/timezones.sqlite --mjolnir-admin " + city_data_dir + "valhalla_tiles/admins.sqlite > " + city_data_dir + "valhalla.json")
print

#build routing tiles
print "Buiding base tiles..."
os.system("valhalla_build_tiles -c valhalla.json extract.pbf")

#tar it up for running the server
print "Buiding tar of tiles..."
os.system("find valhalla_tiles | sort -n | tar cf tiles.tar --no-recursion -T -")

#make some osmlr segments
print "Buiding OSMLR tiles..."
os.system("LD_LIBRARY_PATH=/usr/lib:/usr/local/lib osmlr -m 1 -T osmlr_tiles valhalla.json")

# -j 2 uses two threads for association process (use more or fewer as available cores permit
print "Associate OSMLR tiles..."
os.system("valhalla_associate_segments -t osmlr_tiles -j 2 --config valhalla.json")

# rebuild tar with traffic segement associated tiles
print "Buiding osmlr tile tar..."
os.system("find valhalla_tiles | sort -n | tar rf tiles.tar --no-recursion -T -")

print "Moving built tar tile to " + output_tar
os.rename(city_data_dir + tiles.tar, output_tar)


Downloading: https://s3.amazonaws.com/metro-extracts.mapzen.com/manila_philippines.osm.pbf
Building tile config...

Buiding base tiles...


0